# Machine Learning - Random Forest

In [8]:
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
# pymongo
import pymongo
print('pymongo: {}'.format(pymongo.__version__))

Python: 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
scipy: 1.0.1
numpy: 1.14.2
matplotlib: 2.2.2
pandas: 0.22.0
sklearn: 0.19.1
pymongo: 3.4.0


In [53]:
import warnings
warnings.filterwarnings('ignore')

################################## Load Specific libraries ####################################################

In [58]:
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import preprocessing
from pymongo import MongoClient

# Load dataset

In [55]:
client = MongoClient('localhost', 27017)
db = client.datascience
collection = db.incidents

## Expand the cursor and construct the DataFrame

In [27]:
cursor = collection.find()
incidents = pd.DataFrame(list(cursor))

## Delete the _id


In [28]:
 if no_id:
     del incidents['_id']

 return incidents

## Cargaremos un csv de 2018 (menos datos)

In [23]:
incidents = pd.read_csv("Incidents2017.csv")

## shape. Dimensions of Dataset

In [25]:
print(incidents.shape)

(154773, 13)


## head

In [26]:
print(incidents.head(20))

    IncidntNum                Category  \
0    170670575               EXTORTION   
1    170670575         SECONDARY CODES   
2    170644871          OTHER OFFENSES   
3    170635870          MISSING PERSON   
4    170616046                 ASSAULT   
5    170605817      DISORDERLY CONDUCT   
6    170604449           LARCENY/THEFT   
7    170591103            NON-CRIMINAL   
8    170583332          OTHER OFFENSES   
9    170572868           LARCENY/THEFT   
10   170572307                 ASSAULT   
11   170565437                BURGLARY   
12   170533616                   FRAUD   
13   170527017  SEX OFFENSES, FORCIBLE   
14   170514133          SUSPICIOUS OCC   
15   170465285            NON-CRIMINAL   
16   170451814            NON-CRIMINAL   
17   170451109                   FRAUD   
18   170442158         SECONDARY CODES   
19   170442158                 ASSAULT   

                                             Descript DayOfWeek        Date  \
0                                 ATTE

## Statistical summary. descriptions

In [30]:
print(incidents.describe())

         IncidntNum              X              Y          PdId
count  1.547730e+05  154773.000000  154773.000000  1.547730e+05
mean   1.718220e+08    -122.424253      37.768941  1.718220e+13
std    4.221544e+06       0.026587       0.023565  4.221544e+11
min    1.700554e+07    -122.513642      37.707922  1.700554e+12
25%    1.703404e+08    -122.434308      37.756438  1.703404e+13
50%    1.706792e+08    -122.417488      37.775421  1.706792e+13
75%    1.710148e+08    -122.406844      37.785029  1.710148e+13
max    9.000079e+08    -122.365565      37.819975  9.000079e+13


## Class Distribution

In [34]:
print(incidents.groupby('Category').size())

Category
ARSON                            327
ASSAULT                        13655
BAD CHECKS                        24
BRIBERY                           67
BURGLARY                        5857
DISORDERLY CONDUCT               399
DRIVING UNDER THE INFLUENCE      299
DRUG/NARCOTIC                   3308
DRUNKENNESS                      339
EMBEZZLEMENT                     178
EXTORTION                         63
FAMILY OFFENSES                   44
FORGERY/COUNTERFEITING           509
FRAUD                           2551
GAMBLING                          14
KIDNAPPING                       207
LARCENY/THEFT                  47826
LIQUOR LAWS                       77
LOITERING                         32
MISSING PERSON                  4487
NON-CRIMINAL                   17368
OTHER OFFENSES                 18316
PORNOGRAPHY/OBSCENE MAT            6
PROSTITUTION                     527
RECOVERED VEHICLE                718
ROBBERY                         3351
RUNAWAY                      

## 1.- Create a Validation Dataset

### We will split the loaded dataset into two, 80% of which we will use to train our models and 20% that we will hold back as a validation dataset.

## Split-out validation dataset

In [43]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(incidents, test_size=0.2)

In [44]:
train.shape

(123818, 13)

In [45]:
test.shape

(30955, 13)

In [46]:
weekdays = {'Monday':0., 'Tuesday':1., 'Wednesday':2., 'Thursday': 3., 'Friday':4., 'Saturday':5., 'Sunday':6.}
categories = {c:i for i,c in enumerate(train['Category'].unique())}
cat_rev = {i:c for i,c in enumerate(train['Category'].unique())}
districts = {c:i for i,c in enumerate(train['PdDistrict'].unique())}
dis_rev = {i:c for i,c in enumerate(train['PdDistrict'].unique())}

## Extract features from given information

In [81]:
train['Hour'] = list(map(lambda x: float(int(x.split(' ')[0].split(':')[0])),train.Time))
test['Hour'] = list(map(lambda x: float(int(x.split(' ')[0].split(':')[0])),test.Time))

train['Minute'] = list(map(lambda x: float(int(x.split(' ')[0].split(':')[1])),train.Time))
test['Minute'] = list(map(lambda x: float(int(x.split(' ')[0].split(':')[1])),test.Time))

train['Month'] = list(map(lambda x: float(x.split(' ')[0].split('/')[1]), train.Date))
test['Month'] = list(map(lambda x: float(x.split(' ')[0].split('/')[1]), test.Date))

train['Year'] = list(map(lambda x: float(x.split(' ')[0].split('/')[2])-2003., train.Date))
test['Year'] = list(map(lambda x: float(x.split(' ')[0].split('/')[2])-2003., test.Date))

train['Day'] = list(map(lambda x: float(x.split(' ')[0].split('/')[0]), train.Date))
test['Day'] = list(map(lambda x: float(x.split(' ')[0].split('/')[0]), test.Date))

train['Day_Num'] = [float(weekdays[w]) for w in train.DayOfWeek]
test['Day_Num'] = [float(weekdays[w]) for w in test.DayOfWeek]

train['District_Num'] = [float(districts[t]) for t in train.PdDistrict]
test['District_Num'] = [float(districts[t]) for t in test.PdDistrict]

train['Category_Num'] = [float(categories[t]) for t in train.Category]
test['Category_Num'] = [float(categories[t]) for t in test.Category]

## Center X,Y coordinates

In [71]:
train['X'] = preprocessing.scale(list(map(lambda x: x+122.4194, train.X)))
train['Y'] = preprocessing.scale(list(map(lambda x: x-37.7749, train.Y)))

test['X'] = preprocessing.scale(list(map(lambda x: x+122.4194, test.X)))
test['Y'] = preprocessing.scale(list(map(lambda x: x-37.7749, test.Y)))

## Assign binary value to address by type

In [72]:
def define_address(addr):
    addr_type = 0.
    # Address types:
    #  Intersection: 1
    #  Residence: 0
    if '/' in addr and 'of' not in addr:
        addr_type = 1.
    else:
        add_type = 0.
    return addr_type

## Define address feature

In [73]:
train['Address_Num'] = list(map(define_address, train.Address))
test['Address_Num'] = list(map(define_address, test.Address))

## Feature selection

In [74]:
X_loc = ['X', 'Y', 'District_Num', 'Address_Num']
X_time = ['Minute', 'Hour']
X_date = ['Year','Month', 'Day', 'Day_Num']
X_all = X_loc + X_time + X_date

## Category column we want to predict

In [75]:
y = 'Category_Num'

In [76]:
print(train.head())

        IncidntNum         Category  \
84114    170576484    LARCENY/THEFT   
15060    170103093  SECONDARY CODES   
11226    176024112        VANDALISM   
109196   170766124   OTHER OFFENSES   
134931   170923023         WARRANTS   

                                              Descript  DayOfWeek        Date  \
84114                     PETTY THEFT FROM LOCKED AUTO   Saturday  07/15/2017   
15060                                DOMESTIC VIOLENCE     Sunday  02/05/2017   
11226                    MALICIOUS MISCHIEF, VANDALISM     Friday  01/27/2017   
109196  FALSE PERSONATION TO RECEIVE MONEY OR PROPERTY  Wednesday  09/13/2017   
134931                                  WARRANT ARREST     Sunday  11/12/2017   

         Time PdDistrict      Resolution                      Address  \
84114   16:00   NORTHERN            NONE       400 Block of FULTON ST   
15060   20:42  INGLESIDE  ARREST, BOOKED  1000 Block of VISITACION AV   
11226   20:00    MISSION            NONE      400 Block of 

## Create random forest classifier

In [77]:
clf = RandomForestClassifier(max_features="log2", max_depth=11, n_estimators=24,
                             min_samples_split=1000, oob_score=True)

## Fit prediction

In [78]:
clf.fit(train[X_all], train[y])
pred = clf.predict_proba(test[X_all])